In [1]:
#intalar librerias 
#----------------------------
!pip install selenium
!pip install webdriver_manager 
!pip install datetime
!pip install undetected_chromedriver
!pip install --upgrade webdriver_manager

In [2]:
import pandas as pd # libreria para trabajar con dataframes 
from selenium import webdriver # importar el driver 
from time import sleep # para generar el tiempo de espera entre busquedas 
import lxml
from selenium.webdriver.common.keys import Keys # insertar palabras en las busquedas 
from webdriver_manager.chrome import ChromeDriverManager #importar el instalador del drive 
from datetime import datetime # para detectar la fecha 
from selenium.webdriver.chrome.options import Options #agrega las opciones de navegacion en un driver 
import random # para generar tiempos de espera aleatorios 
import undetected_chromedriver.v2 as uc
import subprocess
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
#Instalar o carga el driver sin tener el ejecutable en la PC
options = Options()#opciones del buscador 
options.add_argument("--incognito")#ventana incognito
options.add_argument("--start-maximized")# inicializar la ventana maximizada 
browser=webdriver.Chrome(ChromeDriverManager().install(),options=options)# abrir el driver con las opciones descritas previamente 
browser.implicitly_wait(4) #cuando hay algun error para seguir probando y de no solucionarse salta al siguiente comando 

#CORREOS -----------------
#davidyacila22@gmail.com    --- osonapo123456
#reyes1999lupita@gmail.com  ---- Lupita1999
#drakevthomas1@gmail.com   ----- Drakethomas1

#EJECUCIÓN -----
ext=['lamolina.edu.pe','gmail.com','hotmail.com']
print("Insertar palabra") 
palabra=input() # insertar la palabra de busqueda 
while(1):
    correo= input("Ingrese el correo para recibir el reporte: ")
    try:
        if correo.split("@")[1] in ext:
            break
        else:
            continue 
    except:
        continue

        
#FUNCIÓN DE BUSQUEDA EN TOTTUS ------
def tottus(palabra):
    try:
        browser.get('https://www.tottus.com.pe') #conexión con la página de Tottus 
        sleep(1) # tiempo de espera de un segundo para cargar la página 
        c_busqueda=browser.find_element("xpath",".//input[@placeholder='¿Qué estás buscando?']") # se busca el casillero para ingresar la palabra a buscar en el HTML
    except: #se repite la accion en caso existan ventanas emergentes
        browser.get('https://www.tottus.com.pe') #conexión con la página de Tottus 
        sleep(1) # tiempo de espera de un segundo para cargar la página 
        c_busqueda=browser.find_element("xpath",".//input[@placeholder='¿Qué estás buscando?']") # se busca el casillero para ingresar la palabra a buscar en el HTML
    
    browser.execute_script("arguments[0].click();", c_busqueda)
    actions = ActionChains(browser) 
    actions.send_keys(palabra)#coloca la palabra en el buscador 
    actions.send_keys(Keys.ENTER)#coloca enter 
    actions.perform() # se coloca la palabra de busqueda (se ingresa el producto)
    sleep(1)  # tiempo de espera para cargar la página 
    lista_prod=[] # se decalara una lista vacia para guardar el nombre de todos los prooductos extraidos 
    while(1):# se genera un bucle para ir de página en página, si se encuentran varios artículos 
        try: # para se ejecute cuando no hay ningun incoveniente con la página, cuando no hay errores 
            pagina=browser.find_element("xpath",".//ul[@class='jsx-1957748114']")# se busca al contenedor de todos los productos por página 
            productos=pagina.find_elements("xpath",".//li") # se busca a los contenedores de cada producto, donde esta el precio, el nombre, la imagen y otros detalles.
            for producto in productos:# se genera un bucle para extraer la información por producto 
                info=[]# se declara una lista vacia para almacenar la información por producto 
                #salen productos sugeridos en la busqueda que no tienen la palabra buscada en la cadena para ello 
                #se hace un validador de busqueda
                if not palabra.lower() in producto.find_element("xpath",".//h2").text.lower(): # se busca la palabra en la cadena extraida para nombre.
                    continue # de no encontrarse la palabra, se continua con el siguiente producto 
                info.append("TOTTUS") # se adiciona el nombre del supermercado 
                info.append(producto.find_element("xpath",".//h2").text)# se busca el nombre del producto y se adiciona a la lista qeu almacena la información por producto 
                try: # si se encuentra el precio del producto que se adicione a la lista de detalles del producto 
                    info.append(float(producto.find_element("xpath",".//span[@class='list price medium cmrPrice ']").text.split(" ")[1]))
                    info.append(producto.find_element("xpath",".//span[@class='list price medium cmrPrice ']").text.split(" ")[2])# se coloca la cantidad - kg o unidades 
                except:
                    continue
                lista_prod.append(info)# adicionar la lista de información del producto 
                print(info)
            browser.find_element("xpath",".//li[@class='next']").click()# avanzar a la siguiente página de productos con un click automatico 
        except: # si no se encuentra el boton para pasar a la siguiente página que se termine el bucle 
            break 
    return(lista_prod) # se retorna la tabla que contiene a todos los productos de tottus 
#tottus(palabra)

#FUNCIÓN DE BUSQUEDA EN METRO ------
def metro(palabra):
    lista_prod=[] # lista vacia para almacenar todos los productos 
    browser.get("https://www.metro.pe/") # se ingresa a la página de metro 
    sleep(1) # tiempo de espera para cargar la página 
    c_busqueda=browser.find_element("xpath","/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input") # se busca el casillero para colocar la palabra de busqueda  
    c_busqueda.send_keys(palabra) # se ingresa a palabra a buscar 
    c_busqueda.send_keys(u'\ue007') # se coloca un enter automático para hacer la busqueda de la palabra 
    sleep(1)# tiempo de espera para cargar la página      
    posicion_y = browser.execute_script("return document.body.scrollHeight")
    posicion_y_nuevo=None
    while posicion_y != posicion_y_nuevo:
        posicion_y=posicion_y_nuevo
        # Desplazar hacia abajo
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")
        # Esperar a que la pagina cargue
        sleep(2)
        # Calcule la nueva altura de desplazamiento y compárela con la última altura de desplazamiento
        posicion_y_nuevo = browser.execute_script("return document.body.scrollHeight")  
    sleep(1)
    try:
        pagina=browser.find_element("xpath",".//div[@class='product-shelf']") # Se extrae el contenedor de todos los productos 
    except: #si no se encuentra al contenedor de productos que se retorne la lista vacia 
        return(lista_prod) # indica que no se encontraron elementos de busqueda 
    productos=pagina.find_elements("xpath",".//li[@layout='19ccd66b-b568-43cb-a106-b52f9796f5cd']") # se extrae todos los contenedores que posee los detalles de cada producto
    print(len(productos))
    for producto in productos: # bucle para extraer la información por producto 
        info=[]# lista para almacenar la información por producto 
        if not palabra.lower() in producto.find_element("xpath",".//a[@class='product-item__name']").text.lower(): # se busca validar  que se encuentre la palabra colocada en la cadena extraida
                #print(producto.find_element("xpath",".//a[@class='product-item__name']").text.lower()) # se imprime el nombre de los productos que no tienen la palabra de busqueda
                continue # de no enconotrarse, que se pase a la siguiente iteración o producto
        info.append("METRO") # agregar el supermercado 
        info.append(producto.find_element("xpath",".//a[@class='product-item__name']").text) # se extrajo el nombre del producto y se adiciona a la lista de detalles 
        precio=producto.find_element("xpath",".//span[@class='product-prices__value product-prices__value--best-price']").text # se busca el precio del prducto y se va validando los diversos formatos 
        if("x" in precio):
            info.append(float(precio.split(" ")[0][2:]))
            info.append(precio.split(" ")[1])
        else:
            precio=precio.replace("S/. ","")
            precio=precio.replace("S/","")
            info.append(float(precio))
            info.append("UN")
        lista_prod.append(info)#se adiciona la información de los productos a la lista general 
        print(info)
    return(lista_prod)# se retorna la lista de productos y su detalle de cada supermercado 

###---PEDRO------------------------------------------------------------------------------------------------------------------------ 
#FUNCIÓN DE BUSQUEDA EN PLAZA VEA  ------
def plazaVea(palabra):
    browser.get('https://www.plazavea.com.pe/') # se ingresa a la página de plaza vea 
    sleep(1)# tiempo de espera para cargar la página 
    c_busqueda=browser.find_element("xpath","/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input") # se busca el casillero para colocar la palabra de busqueda 
    c_busqueda.send_keys(palabra) # se inserta el producto buscado 
    c_busqueda.send_keys(u'\ue007') # se ingresa un enter automático para realizar la busqueda 
    sleep(1)# tiempo de espera para cargar la página 
    lista_prod=[]  # lista vacia para almacenar todos los productos 
    while(1):# se crea el bucle para ir de página en página de busqueda 
        try:#se ejecuta en caso no vayan a ver errores 
            sleep(2)
            pagina=browser.find_element("xpath",".//div[@class='showcase-grid ']") # se busca al contenedor de todos los aproductos 
            productos=pagina.find_elements("xpath",".//div[@class='Showcase__details']") # se extrae todos los contenedores que tienen la información por producto 
            print(len(productos))
            #print(len(productos)) # se imprime la cantidad de productos encontrados 
            for producto in productos: # se busca de producto en producto la información como el precio y el nombre 
                info=[]# lista para almacenar la información por producto 
                try: # se ejecuta cuando no hay errores 
                    if not palabra.lower() in producto.find_element("xpath",".//a[@class='Showcase__name']").text.lower():# se busca l apalabra en la cadena extraida 
                        continue # al no encontrarse se pasa al siguiente contenedor de información 
                    info.append("PLAZA VEA")# se soloca el supermercado 
                    info.append(producto.find_element("xpath",".//a[@class='Showcase__name']").text) # se extrae el nombre del producto 
                    info.append(float(producto.find_element("xpath",".//div[@class='Showcase__salePrice']").text.split(" ")[1]))# se extrae el precio del producto 
                    try:
                        info.append(producto.find_element("xpath",".//div[@class='Showcase__salePrice']").text.split("x ")[1])# se coloca la cantidad - kg o unidades 
                    except:
                        info.append(producto.find_element("xpath",".//div[@class='Showcase__salePrice']").text.split(" ")[2])
                except:
                    continue # que pase al siguiente contenedor en caso de haber error  
                lista_prod.append(info) # se agrega a la lista general de productos 
                print(info)
            browser.find_element("xpath",".//span[@class='pagination__item page-number active']/following-sibling::span").click() # se busca el boton para acceder a la siguiente página y se le da click
        except: #en caso de no encontrarse el boton de siguiente página se corta el bucle iniciado para cada página 
            break 
    return(lista_prod)# se retorna la lista de productos 

lista_tottus= tottus(palabra) # se ejecuta la función de busqueda en el supermercado tottus 
lista_metro= metro(palabra)  # se ejecuta la función de busqueda en el supermercado metro
lista_plaza=plazaVea(palabra) # se ejecuta la función de busqueda en el supermercado plaza Vea 

browser.close()# cerrar el driver
browser.quit()# quitar el driver 



# se convierte en data frame la tabla de precios y productos de cada supermercado 
writer = pd.ExcelWriter('Productos_precio.xlsx', engine='openpyxl') #se crea la hoja excel donde se guardaran las extracciones 
a=pd.DataFrame(lista_metro, columns = ['Supermercado', 'Producto', 'Precio','Cantidad'])
b=pd.DataFrame(lista_tottus, columns = ['Supermercado', 'Producto', 'Precio','Cantidad'])
c=pd.DataFrame(lista_plaza, columns = ['Supermercado', 'Producto', 'Precio','Cantidad'])
frames = [a, b, c]
DF=pd.concat(frames) # se concatena las tablas 
DF=DF.sort_values('Precio')# se ordena de acuerdo al precio
DF=DF.reset_index()# Reiniciar la indexazación 
DF=DF.iloc[:,1:5] #escogiendo columnas de interes 
DF['Producto']=DF['Producto'].str.upper() #se pone todos los nombres en mayúsculas
DF['Cantidad']=DF['Cantidad'].str.upper() #se pone todas las cantidades  en mayúsculas
if len(DF)==0:
    print("no se encontró la busqueda, coloque un producto válido")
else:
    DF.to_excel(writer,sheet_name=palabra,index=False)# convertir en un excel la tabla de productos 
    writer.save()# para guardar el contenido del excel 

    def copy_file(filepath): #funcion para copiar un archivo en la papelera
        cmd = r"gi '{}' | Set-Clipboard".format(filepath) # copia el folder y el contenido 
        subprocess.run(["powershell", "-command", cmd], shell=True)

    def envio(correo): #FUNCION PARA ENVIAR LOS REPORTES A LOS USUARIOS 
        # Colocan las opciones del navegador 
        chrome_options = uc.ChromeOptions()
        chrome_options.add_argument("--disable-extensions") # desabilitar las extenciones del driver 
        chrome_options.add_argument("--disable-popup-blocking") # desabilitar lel bloqueo de cuentas del driver 
        #opciones que permiten entrar a los correos sin problemas 
        chrome_options.add_argument("--profile-directory=Default")
        chrome_options.add_argument("--disable-plugins-discovery")
        chrome_options.add_argument("--incognito") # vetanta de incognito
        chrome_options.add_argument("user_agent=DN")
        chrome_options.add_argument("--start-maximized")
        browser = uc.Chrome(chrome_options=chrome_options)
        browser.delete_all_cookies() # eliminando los cookies del driver 
        browser.implicitly_wait(4) # tiempo de reintento por error 
        browser.get('https://accounts.google.com/AccountChooser/identifier?service=mail&continue=https%3A%2F%2Fmail.google.com%2Fmail%2F&flowName=GlifWebSignIn&flowEntry=AccountChooser') # entrar a la pestaña para colocar correos 
        sleep(1)
        cuadro= browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input') #se busaca el recuadro para colocar el correo 
        user='reyes1999lupita@gmail.com' # se almacena el correo de donde se haran los envios de mensajes 
        password='Lupita1999' # la contraseña 
        cuadro.send_keys(user) # se inserta el correo 
        sleep(2) # tiempo de espera para cargar la página 
        c_pass=browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button/span').click() # darle click a la opción de siguiente
        sleep(1)# tiempo de espera para cargar la página 
        cuadro=  browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div[1]/div/div/div/div/div[1]/div/div[1]/input')# insertar la contraseña 
        cuadro.send_keys(password) # colocar el password 
        sleep(1)# tiempo de espera para cargar la página 
        browser.find_element("xpath",'/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div/div/button/span').click() # darle click para insertar la contraseña 
        sleep(1)# tiempo de espera para cargar la página 
        browser.find_element("xpath",'/html/body/div[7]/div[3]/div/div[2]/div[1]/div[1]/div/div').click()# dar click en "redactar" para colocar un correo 
        sleep(1)# tiempo de espera para cargar la página 
        browser.find_element("xpath",".//input[@class='agP aFw']").send_keys(correo) # ingresar el correo del destinatario 
        browser.find_element("xpath",".//input[@class='aoT']").send_keys("Resumen de precios de productos") # ingresar el asunto 
        browser.find_element("xpath",".//div[@class='Am Al editable LW-avf tS-tW']").click()# ingresar al cuerpo del mensaje 
        ActionChains(browser).send_keys('Estimad@ comprador/a: \n Se le adjunta el reporte de precios de los productos solicitados a los tres supermercados. Tenga en cuenta que están ordenados de menor a mayor por el precio. También se tiene en cuenta las cantidades, es decir, sea kilogramos, gramos o unidades.\n Esperamos que le sea de utilidad.\n Saludos cordiales.').perform()
        
        # enviar el archivo 
        filepath = 'Productos_precio.xlsx' #colocar la dirección de donde se guardara el archivo 
        copy_file(filepath)#copiar el contenido del archivo
        ActionChains(browser).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()#copiar el archivo en el casillero del mensaje 
        sleep(2)
        #browser.find_element("xpath",".//div[@id=':8l']").click()# darle click al boton de enviar
        ActionChains(browser).key_down(Keys.CONTROL).send_keys(Keys.ENTER).key_up(Keys.CONTROL).perform()
        print("se envio la data con exito") # se imprime si se hizo el envio con excito 
        sleep(4)
        browser.close()# cerrar el driver
        browser.quit()# quitar el driver 
    envio(correo)


C:\Users\Lizeth Villanueva\AppData\Local\Temp\ipykernel_1580\3360285298.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser=webdriver.Chrome(ChromeDriverManager().install(),options=options)# abrir el driver con las opciones descritas previamente


Insertar palabra
papaya
Ingrese el correo para recibir el reporte: 20200341@lamolina.edu.pe
7
['METRO', 'Papaya Metro x kg', 5.29, 'x']
['METRO', 'Papaya en Trozos Congelados Natifrut 1kg', 7.49, 'UN']
['METRO', 'Papaya En Trozos Congelada El Frutero Bolsa 1 Kg', 10.5, 'UN']
['METRO', 'Fruta en Trozos Piña, Papaya y Fresa Congelada Natifrut 1kg', 10.87, 'UN']
['METRO', 'Mix Frutero Fresa Papaya Piña x 1 Kg', 13.5, 'UN']
['METRO', 'Fruta en Trozos Piña, Papaya y Sandía Congelada Natifrut 1kg', 10.87, 'UN']
['METRO', 'Origens Comida para Gatos Filete de Atún con Papaya Lata 85 g', 6.4, 'UN']
12
['PLAZA VEA', 'Papaya', 4.49, 'kg']
['PLAZA VEA', 'Mix de Fresa, Papaya y Piña EL FRUTERO Bolsa ...', 13.2, 'un']
['PLAZA VEA', 'Trozos de Papaya EL FRUTERO Bolsa 1Kg', 13.4, 'un']
['PLAZA VEA', 'Papaya en Trozos Congelada FEST Paquete 500g', 7.9, 'un']
['PLAZA VEA', "Ensalada de Piña, Sandía y Papaya BELL'S Bande...", 5.99, 'un']
['PLAZA VEA', "Papaya en Trozos BELL'S Bandeja 300g", 5.39, 'un']
[

In [5]:
DF

,Supermercado,Producto,Precio,Cantidad
0,PLAZA VEA,PAPAYA,4.49,KG
1,METRO,PAPAYA METRO X KG,5.29,X
2,PLAZA VEA,PAPAYA EN TROZOS BELL'S BANDEJA 300G,5.39,UN
3,PLAZA VEA,"ENSALADA DE PIÑA, SANDÍA Y PAPAYA BELL'S BANDE...",5.99,UN
4,METRO,ORIGENS COMIDA PARA GATOS FILETE DE ATÚN CON P...,6.4,UN
5,METRO,PAPAYA EN TROZOS CONGELADOS NATIFRUT 1KG,7.49,UN
6,PLAZA VEA,PAPAYA EN TROZOS CONGELADA FEST PAQUETE 500G,7.9,UN
7,PLAZA VEA,MERMELADA D'MARCO DIETÉTICA DE PIÑA Y PAPAYA F...,8.9,UN
8,METRO,PAPAYA EN TROZOS CONGELADA EL FRUTERO BOLSA 1 KG,10.5,UN
9,METRO,"FRUTA EN TROZOS PIÑA, PAPAYA Y FRESA CONGELADA...",10.87,UN
